In [ ]:
!pip install --quiet gradio pillow requests

import gradio as gr
import requests
import base64
from PIL import Image
import io

GEMINI_API_KEY = "API-KEY"
GEMINI_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

disease_solutions = {
    "Leaf Spot": "⚠ Leaf Spot detected. Use copper-based fungicide and avoid overhead watering.",
    "Rust": "⚠ Rust detected. Remove affected leaves and spray sulfur-based fungicide.",
    "Blight": "⚠ Blight detected. Remove infected plants and apply mancozeb fungicide.",
    "Healthy": "✅ Plant is healthy. Maintain regular watering and sunlight."
}

def analyze_leaf_image(img: Image.Image):
    buffered = io.BytesIO()
    img.convert("RGB").save(buffered, format="JPEG")
    img_base64 = base64.b64encode(buffered.getvalue()).decode()

    prompt = """
    You are an agricultural plant disease detection expert.
    Task:
    1. Identify if this image shows a crop leaf.
    2. If not a leaf, return exactly "NOT_LEAF".
    3. If it is a leaf, check for visible disease.
    4. Classify as one of: Leaf Spot, Rust, Blight, or Healthy.
    5. Return only the disease name.
    """

    payload = {
        "contents": [
            {
                "parts": [
                    {"text": prompt},
                    {
                        "inline_data": {
                            "mime_type": "image/jpeg",
                            "data": img_base64
                        }
                    }
                ]
            }
        ]
    }

    headers = {
        "Content-Type": "application/json",
        "X-goog-api-key": GEMINI_API_KEY
    }

    try:
        response = requests.post(GEMINI_URL, headers=headers, json=payload)
        result = response.json()

        try:
            output_text = result["candidates"][0]["content"]["parts"][0]["text"].strip()
        except:
            return "❌ Error: Could not analyze the image. Please try again."

        if "NOT_LEAF" in output_text.upper():
            return "❌ Error: Uploaded image does not appear to be a leaf. Please upload a clear leaf image."

        output_text = output_text.title()
        if "Leaf Spot" in output_text:
            disease = "Leaf Spot"
        elif "Rust" in output_text:
            disease = "Rust"
        elif "Blight" in output_text:
            disease = "Blight"
        else:
            disease = "Healthy"

        return f"🌿 Prediction: {disease}\nSolution: {disease_solutions[disease]}"

    except Exception as e:
        return f"❌ API Error: {str(e)}"

demo = gr.Interface(
    fn=analyze_leaf_image,
    inputs=gr.Image(type="pil", label="Upload Leaf Image"),
    outputs=gr.Textbox(label="Result"),
    title="🌱 Crop Disease Detection (Gemini 2.0 Flash)",
    description="Upload a leaf image. If not a leaf, an error will appear. If leaf, disease will be detected with a solution.",
    live=False
)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cf501e66d88406e659.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
